In [ ]:
import os
import cv2
import csv
import yaml
from roboflow import Roboflow
from ultralytics import YOLO
from collections import defaultdict
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

rf = Roboflow(api_key="vJRRNV9HbDPFy95m0veX")
project = rf.workspace("object-detection-pxcal").project("pothole-dif-severity-2")
version = project.version(2)
dataset = version.download("yolov11")               

In [ ]:

PADDING_RATIO = 0.2 
MIN_SIZE = 100       
yaml_path = "Pothole-Dif-Severity-2-2/data.yaml"
output_dir = "cropped_dataset_pad2"

with open(yaml_path, 'r') as f:
    class_names = yaml.safe_load(f)['names']

splits = ["train", "test", "valid"]

for split in splits:
    img_dir = os.path.join("Pothole-Dif-Severity-2-2",split, "images")
    label_dir = os.path.join("Pothole-Dif-Severity-2-2",split, "labels")

    for img_file in os.listdir(img_dir):
        if not img_file.lower().endswith((".jpg", ".png")):
            continue

        img_path = os.path.join(img_dir, img_file)
        label_path = os.path.join(label_dir, img_file.rsplit(".", 1)[0] + ".txt")

        if not os.path.exists(label_path):
            continue

        image = cv2.imread(img_path)
        h, w = image.shape[:2]

        with open(label_path, "r") as f:
            lines = f.readlines()

        for idx, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            class_id, x_center, y_center, box_w, box_h = map(float, parts)
            class_name = class_names[int(class_id)]

            x1 = int((x_center - box_w / 2) * w)
            y1 = int((y_center - box_h / 2) * h)
            x2 = int((x_center + box_w / 2) * w)
            y2 = int((y_center + box_h / 2) * h)

            box_w_px = x2 - x1
            box_h_px = y2 - y1

            pad_w = int(box_w_px * PADDING_RATIO)
            pad_h = int(box_h_px * PADDING_RATIO)

            x1_pad = max(0, x1 - pad_w)
            y1_pad = max(0, y1 - pad_h)
            x2_pad = min(w, x2 + pad_w)
            y2_pad = min(h, y2 + pad_h)

            crop = image[y1_pad:y2_pad, x1_pad:x2_pad]
            if crop.size == 0:
                continue

            ch, cw = crop.shape[:2]
            pad_h_extra = max(MIN_SIZE - ch, 0)
            pad_w_extra = max(MIN_SIZE - cw, 0)

            if pad_h_extra > 0 or pad_w_extra > 0:
                crop = cv2.copyMakeBorder(
                    crop,
                    pad_h_extra // 2, pad_h_extra - pad_h_extra // 2,
                    pad_w_extra // 2, pad_w_extra - pad_w_extra // 2,
                    borderType=cv2.BORDER_CONSTANT,
                    value=[0, 0, 0]
                )


            save_dir = os.path.join(output_dir, split, class_name)
            os.makedirs(save_dir, exist_ok=True)

            out_name = f"{img_file.rsplit('.', 1)[0]}_{idx}.jpg"
            cv2.imwrite(os.path.join(save_dir, out_name), crop)